In [1]:
from hig_data.coco import COCOStuffDataset

img_path = '/Users/rupertmenneer/Downloads/coco_val2017_256-sd.zip'
mask_path = '/Users/rupertmenneer/Downloads/coco_val2017_masks_256.zip'
dataset = COCOStuffDataset(img_path, mask_path, 32)


In [1]:
from hig_data.coco import COCOStuffDataset, CocoStuffGraphDataset
import torch
img_path = '/Users/rupertmenneer/Downloads/coco_val2017_256-sd.zip'
mask_path = '/Users/rupertmenneer/Downloads/coco_val2017_masks_256.zip'
coco_graph = CocoStuffGraphDataset(img_path, mask_path, latent_images=True)
data = coco_graph[4]
data

RelaxedHeteroData(
  image=[1, 8, 32, 32],
  mask=[1, 1, 256, 256],
  image_node={
    x=[1024, 1],
    pos=[1024, 2],
  },
  class_node={
    x=[4, 256],
    label=[4],
    pos=[4, 2],
  },
  (class_node, class_edge, class_node)={ edge_index=[2, 12] },
  (class_node, class_to_image, image_node)={ edge_index=[2, 1024] }
)

In [33]:
print(data.mask)
one_hot = torch.nn.functional.one_hot(data.mask, num_classes=256)
print(one_hot.shape)
torch.argmax(one_hot[:,:,0,0,:])

tensor([[[[166,  64,  64,  ...,  87,  87, 255],
          [166,  64,  64,  ...,  87,  87, 255],
          [166,  64,  64,  ...,  87,  87, 255],
          ...,
          [166, 166, 166,  ...,  64,  64, 255],
          [166, 166, 166,  ...,  64,  64, 255],
          [166, 166, 166,  ...,  64,  64, 255]]]])
torch.Size([1, 1, 256, 256, 256])


tensor(166)

In [12]:
torch.argmax(data['class_node'].x, dim=1)

tensor([ 64,  87, 166])

In [2]:
data['class_node'].label

array([ 64,  87, 166], dtype=int16)

In [1]:
from hig_data.coco import COCOStuffGraphPrecomputedDataset
import torch
path = '/Users/rupertmenneer/Downloads/coco_graph_train'
coco_graph = COCOStuffGraphPrecomputedDataset(path)
data = coco_graph[4]
data

Found 118287 complete datapoint in /Users/rupertmenneer/Downloads/coco_graph_train
torch.Size([1024, 1])


RelaxedHeteroData(
  image=[1, 8, 32, 32],
  mask=[1, 1, 256, 256],
  mask_path='/Users/rupertmenneer/Downloads/coco_graph_train/00000004_mask.npy',
  image_node={
    x=[1024, 1],
    pos=[1024, 2],
  },
  class_node={
    x=[7, 183],
    pos=[7, 2],
    label=[7],
  },
  (class_node, class_edge, class_node)={ edge_index=[2, 42] },
  (class_node, class_to_image, image_node)={ edge_index=[2, 944] }
)

In [9]:
from hig_data.coco import CocoStuffGraphDataset
import numpy as np
import os
def create_graph_dataset_and_export(output_dir='/Users/rupertmenneer/Downloads/coco_graph'):

    img_path = '/Users/rupertmenneer/Downloads/coco_val2017_256-sd.zip'
    mask_path = '/Users/rupertmenneer/Downloads/coco_val2017_masks_256.zip'
    coco_graph = CocoStuffGraphDataset(img_path, mask_path, latent_images=True)

    for idx in range(len(coco_graph)):
        image, mask = coco_graph.dataset[idx]
        graph = coco_graph[idx]

        name = coco_graph.dataset._fnames['image'][coco_graph.dataset._raw_idx[idx]]
        raw_id = os.path.splitext(os.path.basename(name))[0].split('-')[-1]

        np.save(os.path.join(output_dir, f'{raw_id}_image.npy'), image)
        np.save(os.path.join(output_dir, f'{raw_id}_mask.npy'), mask)
        np.savez(os.path.join(output_dir, f'{raw_id}_graph'),
                 class_node=graph['class_node'].x,
                 class_pos=graph['class_node'].pos,
                 class_edge=graph['class_node', 'class_edge', 'class_node'].edge_index,
                 class_to_image=graph['class_node', 'class_to_image', 'image_node'].edge_index,
                 )

create_graph_dataset_and_export()


In [ ]:
# from hig_data.coco import COCOStuffDataset, CocoStuffGraphDataset
# import torch
# img_path = '/Users/rupertmenneer/Downloads/coco_val2017_256-sd.zip'
# mask_path = '/Users/rupertmenneer/Downloads/coco_val2017_masks_256.zip'
# coco_graph = CocoStuffGraphDataset(img_path, mask_path, latent_images=True)
# data = coco_graph[4]
# data

In [9]:
128<<13

1048576

In [ ]:
# from hig_data.visualisation import convert_latents_to_pixels
# import PIL
# import numpy as np
# print(data.image.shape)
# img = convert_latents_to_pixels(data.image)

# PIL.Image.fromarray(img)

In [2]:
from torch_geometric.loader import DataLoader as GeoDataLoader
dls = GeoDataLoader(coco_graph, batch_size=32, shuffle=True)
# graph_batch = next(iter(dls))
# graph_batch

# graph_batch['image_node'].pos, graph_batch['class_node'].pos

In [4]:
graph_batch = next(iter(dls))
graph_batch

RelaxedHeteroDataBatch(
  image=[32, 8, 32, 32],
  mask=[32, 1, 256, 256],
  image_node={
    x=[32768, 1],
    pos=[32768, 2],
    batch=[32768],
    ptr=[33],
  },
  class_node={
    x=[297, 256],
    label=[32],
    pos=[297, 2],
    batch=[297],
    ptr=[33],
  },
  (class_node, class_edge, class_node)={ edge_index=[2, 3014] },
  (class_node, class_to_image, image_node)={ edge_index=[2, 32768] }
)

In [9]:
graph_batch['class_node'].label[5]

tensor(0)

In [4]:
for i, graph_batch in enumerate(dls):
    print(i, graph_batch.image.shape)

0 torch.Size([1024, 8, 32, 32])
1 torch.Size([1024, 8, 32, 32])
2 torch.Size([1024, 8, 32, 32])
3 torch.Size([1024, 8, 32, 32])
4 torch.Size([1024, 8, 32, 32])
5 torch.Size([1024, 8, 32, 32])
6 torch.Size([1024, 8, 32, 32])
7 torch.Size([1024, 8, 32, 32])
8 torch.Size([1024, 8, 32, 32])
9 torch.Size([1024, 8, 32, 32])
10 torch.Size([1024, 8, 32, 32])
11 torch.Size([1024, 8, 32, 32])
12 torch.Size([1024, 8, 32, 32])
13 torch.Size([1024, 8, 32, 32])
14 torch.Size([1024, 8, 32, 32])
15 torch.Size([1024, 8, 32, 32])
16 torch.Size([1024, 8, 32, 32])
17 torch.Size([1024, 8, 32, 32])
18 torch.Size([1024, 8, 32, 32])
19 torch.Size([1024, 8, 32, 32])
20 torch.Size([1024, 8, 32, 32])
21 torch.Size([1024, 8, 32, 32])
22 torch.Size([1024, 8, 32, 32])
23 torch.Size([1024, 8, 32, 32])
24 torch.Size([1024, 8, 32, 32])
25 torch.Size([1024, 8, 32, 32])
26 torch.Size([1024, 8, 32, 32])
27 torch.Size([1024, 8, 32, 32])
28 torch.Size([1024, 8, 32, 32])
29 torch.Size([1024, 8, 32, 32])
30 torch.Size([1024,

In [ ]:
graph_batch.detach()

In [8]:
graph_batch['class_node'].label

tensor([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
        0, 0, 0, 0, 0, 0, 0])

In [ ]:
torch.zeros((2, 0), dtype=torch.float32)

In [ ]:
graph_batch['image_node'].pos, graph_batch['class_node'].pos

In [ ]:
# from hig_data.visualisation import save_image_batch_list
# print(graph_batch.image.shape)
# np_img = graph_batch.image[:, :3].cpu().numpy().transpose(0, 2, 3, 1)
# save_image_batch_list([np_img,np_img], row_labels=['img', 'img'], sample_batch_size=8)

In [5]:
from hig_data.visualisation import logging_generate_sample_vis, visualise_het_graph_on_image_batch, plot_array_images
# logging_generate_sample_vis(graph_batch, resize_mask=True)
from training.encoders import StabilityVAEEncoder
vae = StabilityVAEEncoder()
img, decoded = visualise_het_graph_on_image_batch(graph_batch, vae=vae,)
print(img.shape)
plot_array_images(img)


/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Image size: 256
Image size: 256
Image size: 256


KeyboardInterrupt: 

In [ ]:
plot_array_images(decoded)

In [ ]:
from training.networks_edm2_hignn import Precond
precond = Precond(32, 3, gnn_metadata = data.metadata())

In [ ]:
import torch
images = torch.randn(1, 3, 32, 32)
rnd_normal = torch.randn([images.shape[0], 1, 1, 1], device=images.device)
sigma = (rnd_normal).exp()

data = coco_graph[1]
out = precond(images, sigma, data)
out.shape